# Extract MiniROCKET features, and write to ts file

The purpose of this notebook is to extract the MiniROCKET feature set, and write it to train and test .ts files for later exploration

In [3]:
import numpy as np
import pandas as pd
import os
import sys
import time as time
module_path = os.path.abspath(os.path.join('../..'))
from sktime.utils.data_io import load_from_tsfile_to_dataframe

from sklearn.model_selection import train_test_split
from sktime.transformations.panel.rocket import MiniRocket

In [4]:
start = time.time()
X, y = load_from_tsfile_to_dataframe(module_path + '/data/ts_files/crackleNoCrackle_no_rms.ts')
print('Time to fetch X y without compression: ', time.time() - start)

Time to fetch X y without compression:  129.99236798286438


Splitting into training and testing set, with a splitting criteria of 0.2 , 0.8

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Using MiniROCKET to transform the features

In [6]:
start = time.time()
minirocket = MiniRocket()  # by default, MiniRocket uses ~10,000 kernels
minirocket.fit(X_train)
print('Time to fit MiniROCKET: ', time.time() - start)
start = time.time()
X_train_transform = minirocket.transform(X_train)
print('Time to transform X_train: ', time.time() - start)
start = time.time()
X_test_transform = minirocket.transform(X_test)
print('Time to transform X_test: ', time.time() - start)

Time to fit MiniROCKET:  61.64979267120361
Time to transform X_train:  192.94209623336792
Time to transform X_test:  59.12657165527344


Writing the transformed data to train and test files

In [7]:
def write_to_ts(filepath, X, y):
    
    w = open(filepath, 'w+')
    
    w.write('@problemName LungSoundsMiniROCKET \n')
    w.write('@timeStamps false \n')
    w.write('@missing false \n')
    w.write('@univariate true \n')
    w.write('@equalLength true \n')
    w.write(f'@seriesLength {str(len(X.columns))} \n')
    w.write('@classLabel true no_crackle crackle\n')
    w.write('@data \n')
    for (idx, row) in X.iterrows():
        new_row = str((list(row)))[1:-1].replace(' ', '') + ':' + y[idx] + '\n'
        w.write(new_row)


In [8]:
write_to_ts(module_path + '/features/extracted_features_ts_files/lungsound_MiniROCKET_TRAIN.ts', X_train_transform, y_train )
write_to_ts(module_path + '/features/extracted_features_ts_files/lungsound_MiniROCKET_TEST.ts', X_test_transform, y_test )